<strong>Parameters:</strong>
1. Cross Validation nubmer: 5 (both inner and outer)
2. n_estimator: 10, 20, 30, ... , 190, 200
3. max_depth: 5 (I tried 5, 10, 15, ... but 5 was the best)
4. X <- GPS and age, female, hign.educ....

<strong>Result:</strong>
- best max depth: 3


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline

In [3]:
my_data = pd.read_csv('/home/ubuntu/phenotype_and_GPS.csv', header=0)
my_data['KEY'] = my_data['KEY'].astype(str)

#### I placed NAN to mean value... should I replace it?

In [4]:
my_data.fillna(my_data.mean(), inplace=True)

In [5]:
my_data.head()

,KEY,AD,DEPRESSION_SUB,IQ,NEUROTICISM,WORRY_SUB,INSOMNIA,PTSD,SNORING,CP,...,income,married,abcd_site,vol,BMI,NIH_totcomp,NIH_flucomp,CBCL_TotProb,CBCL_Internal,CBCL_External
0,NDARINV003RTV85,-5.780160,0.002583,-2.917600,0.013433,-0.000821,-0.307129,0.000000,0.063292,-2.156561,...,8.0,1.000000,6,-1.050932,20.480539,0.632147,0.605564,-0.789382,-0.731957,-0.589077
1,NDARINV007W6H7B,9.133668,0.002060,0.369931,0.000064,-0.001469,0.438520,0.017525,0.021098,-3.683131,...,10.0,1.000000,22,1.293196,18.234286,1.069812,0.887103,0.268149,0.534371,-0.418541
2,NDARINV00BD7VDC,2.059180,0.006631,-4.261953,0.012193,-0.016278,0.071778,0.010899,0.037388,-4.160142,...,10.0,1.000000,7,1.129021,16.329800,0.632147,1.544026,-0.065808,-0.008341,-0.248005
3,NDARINV00J52GPG,8.591499,0.004259,-2.582771,0.013623,0.000487,-0.328521,0.019801,0.022991,-4.682147,...,6.0,1.522359,17,0.020277,19.061896,0.329044,0.237332,-0.399765,-0.912861,-0.589077
4,NDARINV00NPMHND,3.104598,0.000878,-3.145087,0.013695,-0.013129,0.014641,0.000000,-0.008245,-4.843486,...,8.0,1.000000,17,0.663349,17.663472,-0.680849,-1.365205,1.325680,3.247931,-0.248005


#### CONSTANTS:

In [6]:
k_fold = 5

num_samples = len(my_data)
each_fold_size = int(num_samples/k_fold) +1

print('num_samples: ', num_samples)
print('fold_size: ', each_fold_size)

num_samples:  4567
fold_size:  914


#### PARAMETERS:

In [7]:
X = my_data.iloc[:, 1:35]
X.head()

,AD,DEPRESSION_SUB,IQ,NEUROTICISM,WORRY_SUB,INSOMNIA,PTSD,SNORING,CP,excl23andMe,...,HAPPINESS,age,female,race,high.educ,income,married,abcd_site,vol,BMI
0,-5.780160,0.002583,-2.917600,0.013433,-0.000821,-0.307129,0.000000,0.063292,-2.156561,-0.248034,...,0.002318,131,1,1.0,13.0,8.0,1.000000,6,-1.050932,20.480539
1,9.133668,0.002060,0.369931,0.000064,-0.001469,0.438520,0.017525,0.021098,-3.683131,-1.858655,...,-0.000128,126,0,1.0,19.0,10.0,1.000000,22,1.293196,18.234286
2,2.059180,0.006631,-4.261953,0.012193,-0.016278,0.071778,0.010899,0.037388,-4.160142,-2.382114,...,-0.001354,112,0,1.0,20.0,10.0,1.000000,7,1.129021,16.329800
3,8.591499,0.004259,-2.582771,0.013623,0.000487,-0.328521,0.019801,0.022991,-4.682147,-1.212850,...,0.001577,110,0,5.0,21.0,6.0,1.522359,17,0.020277,19.061896
4,3.104598,0.000878,-3.145087,0.013695,-0.013129,0.014641,0.000000,-0.008245,-4.843486,-1.088557,...,-0.000256,118,1,1.0,16.0,8.0,1.000000,17,0.663349,17.663472


In [8]:
y = my_data['NIH_totcomp']
y.head()

0    0.632147
1    1.069812
2    0.632147
3    0.329044
4   -0.680849
Name: NIH_totcomp, dtype: float64

In [9]:
X = X.to_numpy()
y = y.to_numpy().ravel()

In [10]:
inner_cv = KFold(n_splits = k_fold, shuffle=False, random_state = 123)
outer_cv = KFold(n_splits = k_fold, shuffle=False, random_state = 123)

In [11]:
outerloop = 0

for train_index, test_index in outer_cv.split(X, y):
    outerloop += 1
    print('----------------------------')
    print('Loop: ', outerloop)
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    params = {'n_estimators': np.arange(10, 200, 10),
              'max_depth': [5]}
    
    rf = RandomForestRegressor()
    rf_grid = GridSearchCV(estimator = rf, param_grid = params, cv = inner_cv, scoring='r2', n_jobs=-1, return_train_score=True)
    
    rf_grid.fit(X_train, y_train)
    y_pred_test = rf_grid.predict(X_test)
    #y_pred_train = rf_grid.predict(X_train)
    
    result = pd.DataFrame(rf_grid.cv_results_)[['mean_test_score', 'std_test_score', 'params']]
    print(result)
    print('best train parameter: ', rf_grid.best_params_)
    print('best train score: ', rf_grid.best_score_)
    print('\n')
    print('test score: ', r2_score(y_test, y_pred_test))
    print('----------------------------')

----------------------------
Loop:  1
    mean_test_score  std_test_score                                 params
0          0.180390        0.024770   {'max_depth': 5, 'n_estimators': 10}
1          0.186106        0.024487   {'max_depth': 5, 'n_estimators': 20}
2          0.189756        0.022257   {'max_depth': 5, 'n_estimators': 30}
3          0.191499        0.022479   {'max_depth': 5, 'n_estimators': 40}
4          0.193095        0.024103   {'max_depth': 5, 'n_estimators': 50}
5          0.193249        0.024272   {'max_depth': 5, 'n_estimators': 60}
6          0.193110        0.024760   {'max_depth': 5, 'n_estimators': 70}
7          0.192383        0.026563   {'max_depth': 5, 'n_estimators': 80}
8          0.191108        0.025675   {'max_depth': 5, 'n_estimators': 90}
9          0.192083        0.024919  {'max_depth': 5, 'n_estimators': 100}
10         0.192662        0.024513  {'max_depth': 5, 'n_estimators': 110}
11         0.193169        0.023744  {'max_depth': 5, 'n_estim

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_

    mean_test_score  std_test_score                                 params
0          0.169867        0.028466   {'max_depth': 5, 'n_estimators': 10}
1          0.179002        0.029991   {'max_depth': 5, 'n_estimators': 20}
2          0.176535        0.028647   {'max_depth': 5, 'n_estimators': 30}
3          0.178810        0.029275   {'max_depth': 5, 'n_estimators': 40}
4          0.177322        0.029503   {'max_depth': 5, 'n_estimators': 50}
5          0.175886        0.028418   {'max_depth': 5, 'n_estimators': 60}
6          0.175943        0.028610   {'max_depth': 5, 'n_estimators': 70}
7          0.177041        0.028558   {'max_depth': 5, 'n_estimators': 80}
8          0.177999        0.028912   {'max_depth': 5, 'n_estimators': 90}
9          0.177966        0.028111  {'max_depth': 5, 'n_estimators': 100}
10         0.178165        0.028808  {'max_depth': 5, 'n_estimators': 110}
11         0.178003        0.028330  {'max_depth': 5, 'n_estimators': 120}
12         0.176960      

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_

    mean_test_score  std_test_score                                 params
0          0.172857        0.027181   {'max_depth': 5, 'n_estimators': 10}
1          0.181504        0.027357   {'max_depth': 5, 'n_estimators': 20}
2          0.183284        0.024636   {'max_depth': 5, 'n_estimators': 30}
3          0.183849        0.026357   {'max_depth': 5, 'n_estimators': 40}
4          0.184021        0.025726   {'max_depth': 5, 'n_estimators': 50}
5          0.181860        0.026373   {'max_depth': 5, 'n_estimators': 60}
6          0.182844        0.025329   {'max_depth': 5, 'n_estimators': 70}
7          0.182017        0.025605   {'max_depth': 5, 'n_estimators': 80}
8          0.182295        0.026950   {'max_depth': 5, 'n_estimators': 90}
9          0.183023        0.025574  {'max_depth': 5, 'n_estimators': 100}
10         0.182165        0.025924  {'max_depth': 5, 'n_estimators': 110}
11         0.182408        0.025914  {'max_depth': 5, 'n_estimators': 120}
12         0.182675      

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_

    mean_test_score  std_test_score                                 params
0          0.149948        0.019624   {'max_depth': 5, 'n_estimators': 10}
1          0.159039        0.016069   {'max_depth': 5, 'n_estimators': 20}
2          0.160959        0.015962   {'max_depth': 5, 'n_estimators': 30}
3          0.161708        0.015907   {'max_depth': 5, 'n_estimators': 40}
4          0.163413        0.016797   {'max_depth': 5, 'n_estimators': 50}
5          0.164445        0.016873   {'max_depth': 5, 'n_estimators': 60}
6          0.165070        0.018124   {'max_depth': 5, 'n_estimators': 70}
7          0.166247        0.017477   {'max_depth': 5, 'n_estimators': 80}
8          0.165466        0.015442   {'max_depth': 5, 'n_estimators': 90}
9          0.166327        0.017551  {'max_depth': 5, 'n_estimators': 100}
10         0.165973        0.017384  {'max_depth': 5, 'n_estimators': 110}
11         0.165420        0.018553  {'max_depth': 5, 'n_estimators': 120}
12         0.165767      

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_

    mean_test_score  std_test_score                                 params
0          0.164739        0.030450   {'max_depth': 5, 'n_estimators': 10}
1          0.172673        0.031825   {'max_depth': 5, 'n_estimators': 20}
2          0.173896        0.032397   {'max_depth': 5, 'n_estimators': 30}
3          0.175396        0.031724   {'max_depth': 5, 'n_estimators': 40}
4          0.176944        0.031211   {'max_depth': 5, 'n_estimators': 50}
5          0.176524        0.029837   {'max_depth': 5, 'n_estimators': 60}
6          0.177545        0.028950   {'max_depth': 5, 'n_estimators': 70}
7          0.177957        0.027806   {'max_depth': 5, 'n_estimators': 80}
8          0.178435        0.029228   {'max_depth': 5, 'n_estimators': 90}
9          0.177569        0.029051  {'max_depth': 5, 'n_estimators': 100}
10         0.178235        0.029743  {'max_depth': 5, 'n_estimators': 110}
11         0.178059        0.029480  {'max_depth': 5, 'n_estimators': 120}
12         0.178250      

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_

##### score is R2 score